## 1. Data Preparation 

### Download the data 

In [ ]:
! wget https://www.cs.cmu.edu/~ark/QA-data/data/Question_Answer_Dataset_v1.2.tar.gz

--2021-06-24 13:45:28--  https://www.cs.cmu.edu/~ark/QA-data/data/Question_Answer_Dataset_v1.2.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8254496 (7.9M) [application/x-gzip]
Saving to: ‘Question_Answer_Dataset_v1.2.tar.gz’

Question_Answer_Dat 100%[===================>]   7.87M  1.09MB/s    in 6.7s    

2021-06-24 13:45:35 (1.18 MB/s) - ‘Question_Answer_Dataset_v1.2.tar.gz’ saved [8254496/8254496]



In [ ]:
!mkdir data
!tar -zxf /content/Question_Answer_Dataset_v1.2.tar.gz -C data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
import pandas as pd

s08 = pd.read_csv('/content/data/Question_Answer_Dataset_v1.2/S08/question_answer_pairs.txt',
                          delimiter='\t',engine='python')
s08.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,data/set3/a4
1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,data/set3/a4
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,data/set3/a4
3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.,easy,easy,data/set3/a4
4,Abraham_Lincoln,Did his mother die of pneumonia?,no,easy,medium,data/set3/a4


In [ ]:
s09 = pd.read_csv('/content/data/Question_Answer_Dataset_v1.2/S09/question_answer_pairs.txt',
                          delimiter='\t',engine='python')

s10 = pd.read_csv('/content/data/Question_Answer_Dataset_v1.2/S10/question_answer_pairs.txt',
                          delimiter='\t',engine='python',error_bad_lines=False)

Skipping line 765: '	' expected after '"'
Skipping line 876: '	' expected after '"'
Skipping line 1219: '	' expected after '"'


In [ ]:
s10.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,Alessandro Volta was not a professor of chemis...,easy,easy,data/set4/a10
1,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,No,easy,hard,data/set4/a10
2,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Alessandro Volta did invent the remotely opera...,easy,easy,data/set4/a10
3,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Yes,easy,easy,data/set4/a10
4,Alessandro_Volta,Was Alessandro Volta taught in public schools?,Volta was taught in public schools.,easy,easy,data/set4/a10


In [ ]:
print(len(s08),len(s09),len(s10))
len(s08)+ len(s09)+len(s10)

1715 825 1455


3995

#### Concatenate the datasets
You don't need columns other than question and Answer, so get rid of the rest

In [ ]:
data = pd.concat([s08,s09,s10]).drop(["ArticleTitle","DifficultyFromQuestioner",
                                      "DifficultyFromAnswerer",	"ArticleFile"],axis=1)
print(len(data))
data.head()

3995


,Question,Answer
0,Was Abraham Lincoln the sixteenth President of...,yes
1,Was Abraham Lincoln the sixteenth President of...,Yes.
2,Did Lincoln sign the National Banking Act of 1...,yes
3,Did Lincoln sign the National Banking Act of 1...,Yes.
4,Did his mother die of pneumonia?,no


## 2. Data Processing 

In [ ]:
!pip install spacy --upgrade --quiet
!python -m spacy download en --quiet

     |████████████████████████████████| 12.8MB 222kB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 460kB 27.6MB/s 
     |████████████████████████████████| 624kB 44.5MB/s 
     |████████████████████████████████| 9.1MB 137kB/s 
     |████████████████████████████████| 122kB 51.8MB/s 
2021-06-24 13:46:16.644878: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.7MB 222kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

#### Split the data into training and validation

In [ ]:
from sklearn.model_selection import train_test_split

train_df,valid_df = train_test_split(data,test_size=0.2)
len(train_df),len(valid_df)

(3196, 799)

In [ ]:
train_df.to_csv('train.csv',index=False),valid_df.to_csv('valid.csv',index=False)

(None, None)

#### Load the tokenizer 

In [ ]:
spacy = spacy.load('en_core_web_sm')

In [ ]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy.tokenizer(text)]

In [ ]:
Que = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

Ans = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [ ]:
fields = [('Question', Que),('Answer', Ans)]

#### Create the Datasets needed for creating the DataLoaders 

In [ ]:
from torchtext.legacy import data

train_data, valid_data = data.TabularDataset.splits(
                                        path = '/content',
                                        train = 'train.csv',
                                        validation = 'valid.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True
)

In [ ]:
len(train_data),len(valid_data)

(3196, 799)

#### Look into Dataset structure 

In [ ]:
for i in range(5):
  print(vars(train_data.examples[i]),"\n")

{'Question': ['hard'], 'Answer': ['hard']} 

{'Question': ['how', 'many', 'species', 'and', 'genera', 'does', 'otter', 'have', '?'], 'Answer': ['13', 'species', 'and', '7', 'genera']} 

{'Question': ['some', 'versions', 'of', 'xylophones', 'have', 'resonators', 'made', 'of', 'what', '?'], 'Answer': ['gourds', '.']} 

{'Question': ['when', 'was', 'the', 'principia', 'published', '?'], 'Answer': ['1687']} 

{'Question': ['is', 'or', 'a', 'conception', 'that', 'forms', 'the', 'counterpart', 'of', 'dialect', '?'], 'Answer': []} 



#### Build Vocabulary 

In [ ]:
Que.build_vocab(train_data, min_freq = 2)
Ans.build_vocab(train_data, min_freq = 2)

In [ ]:
print(f"Unique tokens in source (de) vocabulary: {len(Que.vocab)}")

Unique tokens in source (de) vocabulary: 2550


#### Create the DataIterators needed for training and validation

In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator = BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort=False,
    device = device)

In [ ]:
example = next(iter(train_iterator))

In [ ]:
example.Question.shape,example.Answer.shape

(torch.Size([37, 128]), torch.Size([54, 128]))

## 3. Model Building

#### Encoder Architecture

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [ ]:
INPUT_DIM = len(Que.vocab)
EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
DROPOUT = 0.5
enc_test = Encoder(INPUT_DIM,EMB_DIM,HID_DIM,N_LAYERS,DROPOUT).cuda()

#### Sanity and Shape check 

In [ ]:
example.Question.shape

torch.Size([37, 128])

In [ ]:
hidden,cell = enc_test(example.Question)
hidden.shape,cell.shape

(torch.Size([2, 128, 512]), torch.Size([2, 128, 512]))

#### Decoder Architecture

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim,output_dim)
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [ ]:
OUTPUT_DIM = len(Ans.vocab)
EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
DROPOUT = 0.5
dec_test = Decoder(INPUT_DIM,EMB_DIM,HID_DIM,N_LAYERS,DROPOUT).cuda()

### Seq2Seq module integrating Encoder and Decoder 

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]

        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

#### Instantiate the model and pop it onto the GPU 

In [ ]:
INPUT_DIM = len(Que.vocab)
OUTPUT_DIM = len(Ans.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,155,795 trainable parameters


#### Choose Optimizer and Loss fuction Criteria  

In [ ]:
optimizer = optim.AdamW(model.parameters())

In [ ]:
ANS_PAD_IDX =Ans.vocab.stoi[Ans.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = ANS_PAD_IDX)

#### Create Training Loop

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):

        Q = batch.Question
        A= batch.Answer
        
        optimizer.zero_grad()
        
        output = model(Q,A)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        A = A[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output,A)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

#### Create Validation Loop

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            Q = batch.Question
            A = batch.Answer

            output = model(Q,A, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            A = A[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, A)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

#### Run the training loop

In [ ]:
N_EPOCHS = 5
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 5s
	Train Loss: 4.019 | Train PPL:  55.656
	 Val. Loss: 3.735 |  Val. PPL:  41.872
Epoch: 02 | Time: 0m 6s
	Train Loss: 3.953 | Train PPL:  52.084
	 Val. Loss: 3.732 |  Val. PPL:  41.780
Epoch: 03 | Time: 0m 6s
	Train Loss: 3.920 | Train PPL:  50.399
	 Val. Loss: 3.799 |  Val. PPL:  44.657
Epoch: 04 | Time: 0m 6s
	Train Loss: 3.858 | Train PPL:  47.354
	 Val. Loss: 3.762 |  Val. PPL:  43.030
Epoch: 05 | Time: 0m 6s
	Train Loss: 3.819 | Train PPL:  45.549
	 Val. Loss: 3.749 |  Val. PPL:  42.471
